### Import Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./result/KPM_prompt_based_in_group.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')


In [3]:
# df['matched_kp'] = ''
# df['confidence_score'] = ''
# print(df)
# df.to_csv('./result/KPM_prompt_based_in_group.csv', index= False)

In [4]:
print(df)

                                              topic  stance  \
0     Assisted suicide should be a criminal offence       1   
1     Assisted suicide should be a criminal offence       1   
2     Assisted suicide should be a criminal offence       1   
3     Assisted suicide should be a criminal offence       1   
4     Assisted suicide should be a criminal offence       1   
...                                             ...     ...   
7233           The USA is a good country to live in       0   
7234           The USA is a good country to live in       0   
7235           The USA is a good country to live in       0   
7236           The USA is a good country to live in       0   
7237           The USA is a good country to live in       0   

                                               argument  \
0     a cure or treatment may be discovered shortly ...   
1     allow assisted suicide could allow unscrupulou...   
2     Allowing assisted suicide would enable greedy ...   
3     a

In [5]:
prediction_df = pd.read_csv('./result/1by1+predictions_in_group.csv')
print(prediction_df.columns)

Index(['topic', 'stance', 'arguments', 'kep_points',
       'predict_kps(Bert_embedding)', 'predict_kps(avg_embedding)',
       'predict_kps(best_embedding)', 'predict_kps(PCA_embedding)'],
      dtype='object')


### Predict matched KP and calculate score directly from Chatgpt

In [6]:
import openai
import yaml
with open("conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
openai.api_key = credentials['environment_variables']['OPENAI_API_KEY']
model_name = "gpt-4"

In [7]:
def generate_and_eval(topic, argument, kps):
    system_prompt = f"""
        You need to find the best matched kp to the given argument from a list of candidate Kps based on the topic "{topic}",
        and then return this best kp and its confidence score between 0 and 1. 0 represents not match, 1 represents match.
        The result format is as followed: best_kp: .... confidence_score:a number
        """
    rep = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "argument:"+argument+", kps:"+kps}
        ]
    )
    result = rep["choices"][0]["message"]["content"]
    print(result)
    pattern = re.compile(r"best_kp:\s*(.*?)\s*confidence_score:\s*([\d.]+)")
    match = pattern.search(result)

    # Check if both patterns are matched
    if match  :
        kp = match.group(1).strip()
        confidence_score = float(match.group(2))

        # Print or use the extracted information
        print("Key Point:", kp)
        print("Confidence Score:", confidence_score)
        return kp,confidence_score
    else:
        print(f"Error. Regenerating string...")
        return generate_and_eval(topic, argument, kps)




In [10]:
import re
predicted_KPs = []
matching_scores = []
for index,row in df.iloc[7000:].iterrows():
    topic = row['topic']
    stance = row['stance']
    argument = row['argument']
    filtered_df = prediction_df[(prediction_df['topic']==topic) & (prediction_df['stance']== stance)]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps(PCA_embedding)'].values.tolist()
    print(index)
    print(candidate_kps[0])
    # system_prompt = f"""
    #     You need to find the best matched kp to the given argument from a list of candidate Kps based on the topic "{topic}",
    #     and then return this best kp and its confidence score between 0 and 1. 0 represents not match, 1 represents match.
    #     The result format is as followed: best_kp: .... confidence_score:a number
    #     """
    # rep = openai.ChatCompletion.create(
    #     model=model_name,
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": "argument:"+argument+", kps:"+candidate_kps[0]}
    #     ]
    # )
    # result = rep["choices"][0]["message"]["content"]
    # print(result)
    #
    # # Define regular expressions to match the desired patterns
    # # kp_pattern = re.compile(r"best_kp:\s*'([^']+).*?confidence_score:\s*(.*)")
    # # kp_match = kp_pattern.search(result)
    # pattern = re.compile(r"best_kp:\s*(.*?)\s*confidence_score:\s*([\d.]+)")
    # match = pattern.search(result)
    #
    #
    # # Check if both patterns are matched
    # if match  :
    #     kp = match.group(1).strip()
    #     confidence_score = float(match.group(2))
    #
    #     # Print or use the extracted information
    #     print("Key Point:", kp)
    #     print("Confidence Score:", confidence_score)
    # else:
    #     print("No match found.")
    kp, confidence_score = generate_and_eval(topic,argument,candidate_kps[0])
    df.at[index,'matched_kp(1by1)'] = kp
    df.at[index,'confidence_score(1by1)'] = confidence_score

7000
['Government regulation of social media platforms may lead to concerns over invasion of privacy.', 'Government regulation of social media may inhibit freedom of expression.', 'User suggests government regulation might lead to misuse or violations of personal privacy on social media platforms.', 'Lack of government regulation on social media may limit diverse perspectives and hinder the formation of independent opinions.', 'The government regulation of social media platforms is ineffective due to lack of capability in identifying all harmful content.']
best_kp: User suggests government regulation might lead to misuse or violations of personal privacy on social media platforms. confidence_score: 0.85
Key Point: User suggests government regulation might lead to misuse or violations of personal privacy on social media platforms.
Confidence Score: 0.85
7001
['Government regulation of social media platforms may lead to concerns over invasion of privacy.', 'Government regulation of socia

In [11]:
df.to_csv('./result/KPM_prompt_based_in_group.csv', index=False)
print("Add new data successfully!!!!!")


Add new data successfully!!!!!


### Calculate average Confidence Score

In [1]:
import pandas as pd
result_df = pd.read_csv('./result/KPM_prompt_based_in_group.csv')
print(result_df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')


In [2]:
confidence_score = result_df['confidence_score(1by1)'].values.tolist()
print(sum(confidence_score)/len(confidence_score))

0.8610017960762375


### Calculate Tokens

In [1]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt tokenizer for word_tokenize
nltk.download('punkt')

def count_tokens(input_text):
    # Tokenize the input text
    tokens = word_tokenize(input_text)

    # Count the number of tokens
    num_tokens = len(tokens)

    return num_tokens, tokens

# Example usage
input_text = "Hello, how are you today?"
num_tokens, token_list = count_tokens(input_text)

print(f"Input Text: {input_text}")
print(f"Tokens: {token_list}")
print(f"Number of Tokens: {num_tokens}")

Input Text: Hello, how are you today?
Tokens: ['Hello', ',', 'how', 'are', 'you', 'today', '?']
Number of Tokens: 7


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jhluo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd
token_df = pd.read_csv('./result/KPM_prompt_based_in_group.csv')
prediction_df = pd.read_csv('./result/ArgKP21+predictions(v2).csv')
input_text = ''
output_text = ''
for index, row in token_df.iterrows():
    system_prompt = f"""
        You need to find the best matched kp to the given argument from a list of candidate Kps based on the topic "{row['topic']}",
        and then return this best kp and its confidence score between 0 and 1. 0 represents not match, 1 represents match.
        The result format is as followed: best_kp: .... confidence_score:a number
        """
    filtered_df = prediction_df[(prediction_df['topic']==row['topic']) & (prediction_df['stance']== row['stance'])]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps'].values.tolist()
    user_content = "argument:"+row['argument']+", kps:"+candidate_kps[0]
    response_content = row['matched_kp']+str(row['confidence_score'])
    input_text += system_prompt+user_content
    output_text += response_content

In [3]:
input_num_tokens, input_token_list = count_tokens(input_text)
output_num_tokens, output_token_list = count_tokens(output_text)


print(f"Number of Input Tokens: {input_num_tokens}")
print(f"Number of Output Tokens: {output_num_tokens}")

Number of Input Tokens: 1429188
Number of Output Tokens: 132612


### Cost

In [2]:
total_cast = 1429188*30/1000000+132612*60/1000000
print(f"Total cost:{total_cast} $.")

Total cost:50.832359999999994 $.


### Calculate average Confidence Score

In [1]:
import pandas as pd
df = pd.read_csv('./result/KPM_prompt_based_in_group.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp', 'confidence_score'], dtype='object')


In [2]:
score = df['confidence_score'].values.tolist()
print(sum(score)/len(score))

0.8793956894169385
